# Reshaping Dataframes with Dummies

Creating dummy columns is one way to convert a categorical column into numeric columns. If you have a column that has repeated string values, create a new column for each of those values and insert a 1 or a 0 in each new column if corresponds to the original value.

In [1]:
import pandas as pd
import collections
import numpy as np

In [2]:
def tweak_jb(jb):
    
    # A function for getting the unique columns

    def get_uniq_cols(jb):
    
        counter = collections.defaultdict(list)

        for col in sorted(jb.columns):
            period_count = col.count('.')
            if period_count >= 2:
                part_end = 2
        
            else:
                part_end = 1
        
            parts = col.split('.')[:part_end]
            counter['.'.join(parts)].append(col)
    
        uniq_cols = []

        for cols in counter.values():
            uniq_cols.extend(cols)
    
        return uniq_cols
    uniq_cols = get_uniq_cols(jb)
    return(
          jb
    [uniq_cols]
    .rename(columns = lambda col: col.replace('.', '_'))
    .assign(age = lambda df_: df_['age']
                    # Grab the first two numbers in each column
                    .str[0:2]
                    # Convert to float then to the nullable integer
                    .astype('float')
                    .astype('Int64'),
            are_you_datascientist = lambda df_: df_['are_you_datascientist'].replace({'Yes':True, 'No':False, np.nan: 'False'}),
            company_size = lambda df_ : df_['company_size'].replace({'Just me': 1, 'Not sure': np.nan, 'More than 5,000': 5000, '2–10': 2, '11–50': 11, '51–500': 51, '501–1,000': 501,
                                                     '1,001–5,000': 1001}).astype('Int64'),
            # country_live - Convert to categorical.
            country_live = lambda df_: df_['country_live'].astype('category'),
            # employment_status - Fill missing values with 'Other' and convert to categorical.
            employment_status = lambda df_: df_['employment_status'].fillna('Other').astype('category'),
            # is_python_main - Convert to categorical.
            is_python_main = lambda df_: df_['is_python_main'].astype('category'),
            # team_size - Split on en-dash, pull out the first column, replace 'More than 40' with 41, replace
            # values where company_size is NOT 1 with 1, and convert it to a float.
            team_size = lambda df_: df_['team_size'].str.split(r'-', n=1, expand=True)
                                    .iloc[:, 0].replace('More than 40 people', 41)
                                    .where(df_['company_size'] !=1, 1).astype(float),
            # years_of_coding - Replace 'Less than 1 year' with .5, then pull out any numbers with a regular
            # expression, and convert them to floats.
            years_of_coding = lambda df_: df_['years_of_coding']
                                .replace('Less than 1 year ', .5).str.extract(r'(\d+)')
                                .astype('float'),
            # python_years - Replace '_' with '.', then pull out any numbers with a regular expression, and
            # convert them to floats.
            python_years = lambda df_: df_['python_years']
                                        .replace('Less than 1 year ', .5).str.extract(r'(\d+)').astype(float),
            python3_ver = lambda df_: df_['python3_version_most']
                                        .str.replace('_', '.').str.extract(r'(\d\.\d)').astype(float),
            # use_python_most - Replace missing values with 'Unknown'.
            use_python_most = lambda df_: df_['use_python_most'].fillna('Unknown')       
    )
    # Drop the python 2 column
    .drop(columns=['python2_version_most'])
        )

In [3]:
url = '../data/2020-jetbrains-python-survey.csv'

jb = pd.read_csv(url)

jb.head()

C:\Users\INNO\AppData\Local\Temp\ipykernel_43228\2837505772.py:3: DtypeWarning: Columns (152) have mixed types. Specify dtype option on import or set low_memory=False.
  jb = pd.read_csv(url)


,is.python.main,other.lang.None,other.lang.Java,other.lang.JavaScript,other.lang.C/C++,other.lang.PHP,other.lang.C#,other.lang.Ruby,other.lang.Bash / Shell,other.lang.Objective-C,...,job.role.Technical support,job.role.Data analyst,job.role.Business analyst,job.role.Team lead,job.role.Product manager,job.role.CIO / CEO / CTO,job.role.Systems analyst,job.role.Other,age,country.live
0,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bash / Shell,NaN,...,NaN,NaN,Business analyst,NaN,NaN,NaN,NaN,NaN,30–39,NaN
1,Yes,NaN,Java,JavaScript,NaN,NaN,C#,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21–29,India
2,Yes,NaN,NaN,NaN,C/C++,NaN,NaN,NaN,Bash / Shell,NaN,...,Technical support,Data analyst,NaN,Team lead,NaN,NaN,NaN,NaN,30–39,United States
3,Yes,NaN,NaN,JavaScript,NaN,NaN,NaN,NaN,Bash / Shell,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Yes,NaN,Java,JavaScript,C/C++,NaN,NaN,NaN,Bash / Shell,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21–29,Italy


In [4]:
job = (
    jb
    # Filter the df using a regex
    .filter(regex=r'job.role*')
    # Return a df with null values and replace non-null values with 1
    .where(jb.isna(), 1)
    # fill null values with 0
    .fillna(0)
    # scan along an axis and report the index (orcolumn) where the maximum value is found.
    .idxmax(axis='columns')
    # Remove the string job.role
    .str.replace('job.role.', '', regex=False)
)

> The job series now looks like a column with categorical data. This is the type of column we
usually want to convert into dummy columns

In [5]:
dum = pd.get_dummies(job)

dum.head()

,Architect,Business analyst,CIO / CEO / CTO,DBA,Data analyst,Developer / Programmer,Other,Product manager,QA engineer,Systems analyst,Team lead,Technical support,Technical writer
0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,0,0,0


# Reshaping By Pivoting and Grouping

In [6]:
jb.head()

,is.python.main,other.lang.None,other.lang.Java,other.lang.JavaScript,other.lang.C/C++,other.lang.PHP,other.lang.C#,other.lang.Ruby,other.lang.Bash / Shell,other.lang.Objective-C,...,job.role.Technical support,job.role.Data analyst,job.role.Business analyst,job.role.Team lead,job.role.Product manager,job.role.CIO / CEO / CTO,job.role.Systems analyst,job.role.Other,age,country.live
0,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bash / Shell,NaN,...,NaN,NaN,Business analyst,NaN,NaN,NaN,NaN,NaN,30–39,NaN
1,Yes,NaN,Java,JavaScript,NaN,NaN,C#,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21–29,India
2,Yes,NaN,NaN,NaN,C/C++,NaN,NaN,NaN,Bash / Shell,NaN,...,Technical support,Data analyst,NaN,Team lead,NaN,NaN,NaN,NaN,30–39,United States
3,Yes,NaN,NaN,JavaScript,NaN,NaN,NaN,NaN,Bash / Shell,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Yes,NaN,Java,JavaScript,C/C++,NaN,NaN,NaN,Bash / Shell,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21–29,Italy


In [7]:
jb_2 = tweak_jb(jb)

(
    jb_2
    .pivot_table(index='country_live', columns='employment_status',
                 values='age', aggfunc='min')
)

employment_status,Freelancer (a person pursuing a profession without a long-term commitment to any one employer),Fully employed by a company / organization,Other,Partially employed by a company / organization,Retired,"Self-employed (a person earning income directly from one's own business, trade, or profession)",Student,Working student
country_live,,,,,,,,
Algeria,21,18,30,18,60,18,18,21
Argentina,18,18,40,18,50,21,18,18
Armenia,30,18,60,18,40,21,<NA>,<NA>
Australia,21,18,18,21,18,18,18,18
Austria,21,21,30,18,60,30,18,18
...,...,...,...,...,...,...,...,...
United States,18,18,18,18,30,18,18,18
Uruguay,30,21,<NA>,21,<NA>,30,18,<NA>
Uzbekistan,18,21,18,21,<NA>,21,18,21


It turns out that we can use the **pd.crosstab** function as well. Because this is a function, we need to provide the data as series rather than the column names:

In [8]:
(
    pd.crosstab(index=jb_2['country_live'],               
                columns=jb_2['employment_status'],
                values=jb_2['age'],
                aggfunc='min'
                )
)

employment_status,Freelancer (a person pursuing a profession without a long-term commitment to any one employer),Fully employed by a company / organization,Other,Partially employed by a company / organization,Retired,"Self-employed (a person earning income directly from one's own business, trade, or profession)",Student,Working student
country_live,,,,,,,,
Algeria,21,18,30,18,60,18,18,21
Argentina,18,18,40,18,50,21,18,18
Armenia,30,18,60,18,40,21,<NA>,<NA>
Australia,21,18,18,21,18,18,18,18
Austria,21,21,30,18,60,30,18,18
...,...,...,...,...,...,...,...,...
United States,18,18,18,18,30,18,18,18
Uruguay,30,21,<NA>,21,<NA>,30,18,<NA>
Uzbekistan,18,21,18,21,<NA>,21,18,21


In [9]:
# Using Groupby

(
    jb_2
    # Groupby the country_live and employment_status
    .groupby(['country_live', 'employment_status'])
    #Select the age column
    ['age']
    #Get the min
    .mean()
    # Unstack
    .unstack()
)

employment_status,Freelancer (a person pursuing a profession without a long-term commitment to any one employer),Fully employed by a company / organization,Other,Partially employed by a company / organization,Retired,"Self-employed (a person earning income directly from one's own business, trade, or profession)",Student,Working student
country_live,,,,,,,,
Algeria,33.285714,28.344828,43.333333,25.181818,60.0,28.411765,20.4375,21.0
Argentina,34.954545,30.209524,40.0,30.25,55.0,29.571429,22.913043,23.2
Armenia,30.0,23.842105,60.0,24.0,40.0,24.0,<NA>,<NA>
Australia,30.818182,33.074468,33.1,32.0,48.285714,39.21875,24.828571,22.470588
Austria,35.857143,31.318182,30.0,29.0,60.0,34.615385,20.842105,23.8125
...,...,...,...,...,...,...,...,...
United States,34.318471,32.343223,35.534884,25.894737,54.920635,38.446512,21.947791,22.89375
Uruguay,30.0,28.909091,<NA>,36.2,<NA>,30.0,20.0,<NA>
Uzbekistan,36.0,22.125,29.0,21.0,<NA>,30.5,19.0,21.0


## Custom Aggregations

What is the percentage of Emacs users by country?

In [10]:
def per_emacs(ser):
    return ser.str.contains('Emacs').mean() * 100

In [11]:
(
    jb_2.pivot_table(index='country_live', 
                     values='ide_main',
                     aggfunc=per_emacs
                     )
)

,ide_main
country_live,
Algeria,0.000000
Argentina,4.324324
Armenia,0.000000
Australia,2.970297
Austria,1.630435
...,...
United States,4.164498
Uruguay,0.000000
Uzbekistan,0.000000


## Multiple Aggregations

What is the minimum and maximum age for each country?

In [12]:
(
    jb_2
    # Groupby the country
    .groupby(['country_live'])
    # Select the age column
    ['age']
    # Perform the aggregation
    .agg(['min', 'max'])
)

,min,max
country_live,,
Algeria,18,60
Argentina,18,60
Armenia,18,60
Australia,18,60
Austria,18,60
...,...,...
United States,18,60
Uruguay,18,60
Uzbekistan,18,60


In [13]:
# Answering the same question with a pivot_table

(
    jb_2.pivot_table(index='country_live',
                     values='age',
                     aggfunc=(min, max)
                    )
)

,max,min
country_live,,
Algeria,60,18
Argentina,60,18
Armenia,60,18
Australia,60,18
Austria,60,18
...,...,...
United States,60,18
Uruguay,60,18
Uzbekistan,60,18


## Per Column Aggregations

In [14]:
(
    jb_2
    .pivot_table(index='country_live',
                 aggfunc=(min, max)
                 )
)

C:\Users\INNO\AppData\Local\Temp\ipykernel_43228\1140494420.py:2: FutureWarning: ['are_you_datascientist', 'bigdata_Apache Beam', 'bigdata_Apache Flink', 'bigdata_Apache Hadoop/MapReduce', 'bigdata_Apache Hive', 'bigdata_Apache Kafka', 'bigdata_Apache Samza', 'bigdata_Apache Spark', 'bigdata_Apache Tez', 'bigdata_ClickHouse', 'bigdata_Dask', 'bigdata_None', 'bigdata_Other', 'ci_AppVeyor', 'ci_Bamboo', 'ci_CircleCI', 'ci_CruiseControl', 'ci_Gitlab CI', 'ci_Jenkins / Hudson', 'ci_None', 'ci_Other', 'ci_TeamCity', 'ci_Travis CI', 'cloud_platform_AWS', 'cloud_platform_DigitalOcean', 'cloud_platform_Google Cloud Platform', 'cloud_platform_Heroku', 'cloud_platform_Linode', 'cloud_platform_Microsoft Azure', 'cloud_platform_None', 'cloud_platform_OpenShift', 'cloud_platform_OpenStack', 'cloud_platform_Other', 'cloud_platform_PythonAnywhere', 'cloud_platform_Rackspace', 'configuration_management_Ansible', 'configuration_management_Chef', 'configuration_management_Custom solution', 'configuratio

age     company_size     nps_main_ide      python3_ver       \
              max min          max min          max  min         max  min   
country_live                                                                
Algeria        60  18         5000   1         10.0  0.0         3.9  3.5   
Argentina      60  18         5000   1         10.0  3.0         3.9  3.5   
Armenia        60  18         5000   1         10.0  5.0         3.9  3.5   
Australia      60  18         5000   1         10.0  0.0         3.9  3.5   
Austria        60  18         5000   1         10.0  3.0         3.9  3.5   
...            ..  ..          ...  ..          ...  ...         ...  ...   
United States  60  18         5000   1         10.0  0.0         3.9  3.5   
Uruguay        60  18         5000   2         10.0  5.0         3.9  3.5   
Uzbekistan     60  18         5000   1         10.0  2.0         3.9  3.6   
Venezuela      60  18         5000   1         10.0  1.0         3.9  3.5   
Viet Nam       60  18         5000   1         10.0  5.0         3.9  3.5   

              python_years      team_size       use_python_most  \
                       max  min       max  min              max   
country_live                                                      
Algeria               11.0  1.0      21.0  1.0  Web development   
Argentina             11.0  1.0      21.0  1.0  Web development   
Armenia                6.0  1.0      41.0  1.0  Web development   
Australia             11.0  1.0      41.0  1.0  Web development   
Austria               11.0  1.0      13.0  1.0  Web development   
...                    ...  ...       ...  ...              ...   
United States         11.0  1.0      41.0  1.0  Web development   
Uruguay               11.0  1.0      41.0  1.0  Web development   
Uzbekistan             6.0  1.0       2.0  1.0  Web development   
Venezuela             11.0  1.0      21.0  1.0  Web development   
Viet Nam              11.0  1.0      41.0  1.0  Web development   

                                 years_of_coding       
                             min             max  min  
country_live                                           
Algeria        Computer graphics            11.0  1.0  
Argentina          Data analysis            11.0  1.0  
Armenia        Computer graphics            11.0  1.0  
Australia      Computer graphics            11.0  1.0  
Austria        Computer graphics            11.0  1.0  
...                          ...             ...  ...  
United States  Computer graphics            11.0  1.0  
Uruguay            Data analysis            11.0  1.0  
Uzbekistan     Computer graphics             6.0  1.0  
Venezuela          Data analysis            11.0  1.0  
Viet Nam       Computer graphics            11.0  1.0  

[76 rows x 16 columns]

In [15]:
## Group by Version

(
    jb_2
    # Group by country_live
    .groupby(['country_live'])
    # Aggregation for each column
    .agg(['min', 'max'])
)

C:\Users\INNO\AppData\Local\Temp\ipykernel_43228\1539515022.py:4: FutureWarning: ['are_you_datascientist', 'bigdata_Apache Beam', 'bigdata_Apache Flink', 'bigdata_Apache Hadoop/MapReduce', 'bigdata_Apache Hive', 'bigdata_Apache Kafka', 'bigdata_Apache Samza', 'bigdata_Apache Spark', 'bigdata_Apache Tez', 'bigdata_ClickHouse', 'bigdata_Dask', 'bigdata_None', 'bigdata_Other', 'ci_AppVeyor', 'ci_Bamboo', 'ci_CircleCI', 'ci_CruiseControl', 'ci_Gitlab CI', 'ci_Jenkins / Hudson', 'ci_None', 'ci_Other', 'ci_TeamCity', 'ci_Travis CI', 'cloud_platform_AWS', 'cloud_platform_DigitalOcean', 'cloud_platform_Google Cloud Platform', 'cloud_platform_Heroku', 'cloud_platform_Linode', 'cloud_platform_Microsoft Azure', 'cloud_platform_None', 'cloud_platform_OpenShift', 'cloud_platform_OpenStack', 'cloud_platform_Other', 'cloud_platform_PythonAnywhere', 'cloud_platform_Rackspace', 'configuration_management_Ansible', 'configuration_management_Chef', 'configuration_management_Custom solution', 'configuratio

age     company_size       nps_main_ide       python_years  \
              min max          min   max          min   max          min   
country_live                                                               
Algeria        18  60            1  5000          0.0  10.0          1.0   
Argentina      18  60            1  5000          3.0  10.0          1.0   
Armenia        18  60            1  5000          5.0  10.0          1.0   
Australia      18  60            1  5000          0.0  10.0          1.0   
Austria        18  60            1  5000          3.0  10.0          1.0   
...            ..  ..          ...   ...          ...   ...          ...   
United States  18  60            1  5000          0.0  10.0          1.0   
Uruguay        18  60            2  5000          5.0  10.0          1.0   
Uzbekistan     18  60            1  5000          2.0  10.0          1.0   
Venezuela      18  60            1  5000          1.0  10.0          1.0   
Viet Nam       18  60            1  5000          5.0  10.0          1.0   

                    team_size          use_python_most                   \
                max       min   max                min              max   
country_live                                                              
Algeria        11.0       1.0  21.0  Computer graphics  Web development   
Argentina      11.0       1.0  21.0      Data analysis  Web development   
Armenia         6.0       1.0  41.0  Computer graphics  Web development   
Australia      11.0       1.0  41.0  Computer graphics  Web development   
Austria        11.0       1.0  13.0  Computer graphics  Web development   
...             ...       ...   ...                ...              ...   
United States  11.0       1.0  41.0  Computer graphics  Web development   
Uruguay        11.0       1.0  41.0      Data analysis  Web development   
Uzbekistan      6.0       1.0   2.0  Computer graphics  Web development   
Venezuela      11.0       1.0  21.0      Data analysis  Web development   
Viet Nam       11.0       1.0  41.0  Computer graphics  Web development   

              years_of_coding       python3_ver       
                          min   max         min  max  
country_live                                          
Algeria                   1.0  11.0         3.5  3.9  
Argentina                 1.0  11.0         3.5  3.9  
Armenia                   1.0  11.0         3.5  3.9  
Australia                 1.0  11.0         3.5  3.9  
Austria                   1.0  11.0         3.5  3.9  
...                       ...   ...         ...  ...  
United States             1.0  11.0         3.5  3.9  
Uruguay                   1.0  11.0         3.5  3.9  
Uzbekistan                1.0   6.0         3.6  3.9  
Venezuela                 1.0  11.0         3.5  3.9  
Viet Nam                  1.0  11.0         3.5  3.9  

[76 rows x 16 columns]

Sometimes, we want to specify aggregations per column. With both the `.pivot_table` and
`.groupby` methods, we can provide a dictionary mapping a column to an aggregation function or a
list of aggregation functions.

Assume your boss asked: ”What are the minimum and maximum ages and the average team
size for each country?”

In [16]:
(
    jb_2
    # Group by country_live
    .groupby(['country_live'])
    # Aggregations
    .agg({
        'team_size': ['min', 'max'],
        'age': ['min']
        })
)

team_size       age
                    min   max min
country_live                     
Algeria             1.0  21.0  18
Argentina           1.0  21.0  18
Armenia             1.0  41.0  18
Australia           1.0  41.0  18
Austria             1.0  13.0  18
...                 ...   ...  ..
United States       1.0  41.0  18
Uruguay             1.0  41.0  18
Uzbekistan          1.0   2.0  18
Venezuela           1.0  21.0  18
Viet Nam            1.0  41.0  18

[76 rows x 3 columns]

In [17]:
(
    jb_2
    .pivot_table(index='country_live', 
                 aggfunc={'age': ['min', 'max'],
                          'team_size': ['min']
                        })
)

age     team_size
              max min       min
country_live                   
Algeria        60  18       1.0
Argentina      60  18       1.0
Armenia        60  18       1.0
Australia      60  18       1.0
Austria        60  18       1.0
...            ..  ..       ...
United States  60  18       1.0
Uruguay        60  18       1.0
Uzbekistan     60  18       1.0
Venezuela      60  18       1.0
Viet Nam       60  18       1.0

[76 rows x 3 columns]

In [18]:
# Named Aggregations
(
    jb_2
    .groupby(['country_live'])
    .agg(age_min=('age', 'min'),
         age_max=('age', 'max'),
         team_size_mean=('team_size', 'mean')
        )
)

,age_min,age_max,team_size_mean
country_live,,,
Algeria,18,60,2.428571
Argentina,18,60,3.192053
Armenia,18,60,6.076923
Australia,18,60,2.710884
Austria,18,60,2.448000
...,...,...,...
United States,18,60,3.391337
Uruguay,18,60,4.692308
Uzbekistan,18,60,1.160000


## Grouping By Hierarchy

Suppose your boss asked about minimum and maximum age for each country and editor. We want to have both the country and the editor in the index.

In [19]:
(
    jb_2
    # Insert a Pivot Table
    .pivot_table(index=['country_live', 'ide_main'],
                 values=['age'],
                 aggfunc=[min, max]
                 )
)

min max
                                          age age
country_live ide_main                            
Algeria      Atom                          18  60
             Eclipse + Pydev               18  30
             IDLE                          18  50
             IntelliJ IDEA                 21  21
             Jupyter Notebook              21  30
...                                        ..  ..
Viet Nam     PyCharm Community Edition     21  30
             PyCharm Professional Edition  18  30
             Spyder                        21  21
             VS Code                       18  30
             Vim                           18  40

[1059 rows x 2 columns]

In [20]:
# Group by Version

(
    jb_2
    # Group by country_live and ide_main and dropna
    .groupby(['country_live', 'ide_main'], observed=True)
    # Select the age column
    [['age']]
    # Aggregate function
    .agg([min, max])
    .swaplevel(axis='columns')
)

min max
                                                age age
country_live       ide_main                            
India              Atom                          18  40
                   Eclipse + Pydev               18  60
                   Emacs                         21  50
                   IDLE                          18  60
                   IntelliJ IDEA                 18  30
...                                              ..  ..
Dominican Republic PyCharm Community Edition     21  21
                   PyCharm Professional Edition  21  30
                   Sublime Text                  21  40
                   VS Code                       18  50
                   Vim                           21  21

[1059 rows x 2 columns]

In [21]:
def even_grouper(idx):
    return 'odd' if idx % 2 else 'even'



(
    jb_2
    .pivot_table(
                    index=even_grouper, 
                    aggfunc='size'
                )
)

even    27231
odd     27231
dtype: int64

## More Aggregations

Let’s assume we are still looking at the JetBrains dataset and wanted to add a new column, the count
of responses from a country.

In [22]:
(
    jb_2
    # Group by country_live
    .groupby(['country_live'])
    # Select the age column
    [['age']]
    # unstack
    .transform('count')
)

,age
0,76
1,2799
2,3972
3,76
4,441
...,...
54457,935
54458,76
54459,935
54460,452


In [23]:
# Filtering Group by Objects

(
    jb_2
    # Group by the country_live
    .groupby('country_live')
    # Filter the groupby object for size >=60.5
    .filter(lambda gb_ : gb_[['country_live']].size >= 60.5)
)

,age,are_you_datascientist,bigdata_Apache Beam,bigdata_Apache Flink,bigdata_Apache Hadoop/MapReduce,bigdata_Apache Hive,bigdata_Apache Kafka,bigdata_Apache Samza,bigdata_Apache Spark,bigdata_Apache Tez,...,web_frameworks_FastAPI,web_frameworks_Flask,web_frameworks_Hug,web_frameworks_None,web_frameworks_Other,web_frameworks_Pyramid,web_frameworks_Tornado,web_frameworks_web2py,years_of_coding,python3_ver
1,21,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Tornado,NaN,3.0,3.6
2,30,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Flask,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3.6
4,21,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Flask,NaN,NaN,NaN,NaN,NaN,NaN,1.0,3.8
5,21,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,FastAPI,Flask,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3.8
10,21,False,NaN,NaN,NaN,NaN,Apache Kafka,NaN,NaN,NaN,...,NaN,Flask,NaN,NaN,NaN,NaN,NaN,NaN,1.0,3.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54456,30,False,NaN,NaN,Apache Hadoop/MapReduce,Apache Hive,NaN,NaN,Apache Spark,NaN,...,NaN,Flask,NaN,NaN,NaN,NaN,NaN,NaN,6.0,3.6
54457,21,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,FastAPI,Flask,NaN,NaN,NaN,NaN,NaN,NaN,1.0,3.6
54459,21,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,FastAPI,Flask,NaN,NaN,NaN,NaN,NaN,NaN,6.0,3.7
54460,30,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3.7


> The `.filter` method accepts a function that takes the current group. If the function returns True (it must return a scalar, not a series or dataframe), the rows are kept for the result.

## Cross-Tabulation Deep Dive

In [24]:
pd.crosstab(index=jb_2['country_live'], columns=jb_2['age'])

age,18,21,30,40,50,60
country_live,,,,,,
Algeria,18,31,21,10,3,4
Argentina,12,67,67,34,11,4
Armenia,2,19,7,1,0,2
Australia,26,96,150,87,38,20
Austria,10,54,82,26,15,1
...,...,...,...,...,...,...
United States,199,1144,1378,650,381,220
Uruguay,1,9,18,4,0,1
Uzbekistan,8,18,2,2,0,1


> Both `.pivot_table` and `crosstab` have a parameter that will put in a column and row at the  
margins right and bottom respectively that summarize the data:    

In [25]:
pd.crosstab(index=jb_2['country_live'], columns=jb_2['age'], margins=True)

age,18,21,30,40,50,60,All
country_live,,,,,,,
Algeria,18,31,21,10,3,4,87
Argentina,12,67,67,34,11,4,195
Armenia,2,19,7,1,0,2,31
Australia,26,96,150,87,38,20,417
Austria,10,54,82,26,15,1,188
...,...,...,...,...,...,...,...
Uruguay,1,9,18,4,0,1,33
Uzbekistan,8,18,2,2,0,1,31
Venezuela,9,30,12,8,5,2,66


## Normalizing Results

In [26]:
pd.crosstab(index=jb_2['country_live'], columns=jb_2['age'], normalize=True, margins=True)

age,18,21,30,40,50,60,All
country_live,,,,,,,
Algeria,0.000728,0.001254,0.000849,0.000404,0.000121,0.000162,0.003519
Argentina,0.000485,0.002710,0.002710,0.001375,0.000445,0.000162,0.007887
Armenia,0.000081,0.000769,0.000283,0.000040,0.000000,0.000081,0.001254
Australia,0.001052,0.003883,0.006067,0.003519,0.001537,0.000809,0.016867
Austria,0.000404,0.002184,0.003317,0.001052,0.000607,0.000040,0.007604
...,...,...,...,...,...,...,...
Uruguay,0.000040,0.000364,0.000728,0.000162,0.000000,0.000040,0.001335
Uzbekistan,0.000324,0.000728,0.000081,0.000081,0.000000,0.000040,0.001254
Venezuela,0.000364,0.001213,0.000485,0.000324,0.000202,0.000081,0.002670


In [27]:
# Normalize down the columns
pd.crosstab(index=jb_2['country_live'], columns=jb_2['age'], normalize='columns')

age,18,21,30,40,50,60
country_live,,,,,,
Algeria,0.007031,0.003198,0.002799,0.003323,0.002187,0.006838
Argentina,0.004687,0.006911,0.008931,0.011299,0.008017,0.006838
Armenia,0.000781,0.001960,0.000933,0.000332,0.000000,0.003419
Australia,0.010156,0.009902,0.019995,0.028913,0.027697,0.034188
Austria,0.003906,0.005570,0.010930,0.008641,0.010933,0.001709
...,...,...,...,...,...,...
United States,0.077734,0.117999,0.183684,0.216019,0.277697,0.376068
Uruguay,0.000391,0.000928,0.002399,0.001329,0.000000,0.001709
Uzbekistan,0.003125,0.001857,0.000267,0.000665,0.000000,0.001709


In [28]:
# Normalize down the rows
pd.crosstab(index=jb_2['country_live'], columns=jb_2['age'], normalize='index')

age,18,21,30,40,50,60
country_live,,,,,,
Algeria,0.206897,0.356322,0.241379,0.114943,0.034483,0.045977
Argentina,0.061538,0.343590,0.343590,0.174359,0.056410,0.020513
Armenia,0.064516,0.612903,0.225806,0.032258,0.000000,0.064516
Australia,0.062350,0.230216,0.359712,0.208633,0.091127,0.047962
Austria,0.053191,0.287234,0.436170,0.138298,0.079787,0.005319
...,...,...,...,...,...,...
United States,0.050101,0.288016,0.346928,0.163646,0.095921,0.055388
Uruguay,0.030303,0.272727,0.545455,0.121212,0.000000,0.030303
Uzbekistan,0.258065,0.580645,0.064516,0.064516,0.000000,0.032258


## Hierarchical Columns with Cross Tabulations

In [29]:
(
    pd.crosstab(index=[jb_2['country_live'], jb_2['age']], 
                columns=[jb_2['use_python_most'], jb_2['python3_version_most']]).loc[['United States']]
)

use_python_most        Computer graphics                                   \
python3_version_most Python 3_5 or lower Python 3_6 Python 3_7 Python 3_8   
country_live  age                                                           
United States 18                       0          0          0          2   
              21                       0          0          1          3   
              30                       0          0          0          1   
              40                       0          0          1          3   
              50                       0          0          1          2   
              60                       0          0          1          0   

use_python_most                       Data analysis                        \
python3_version_most Python 3_9 Python 3_5 or lower Python 3_6 Python 3_7   
country_live  age                                                           
United States 18              0                   0          1          5   
              21              2                   2         25         74   
              30              0                   3         34         83   
              40              0                   0         18         39   
              50              0                   2          7         33   
              60              1                   1          2         12   

use_python_most                             ...             Unknown  \
python3_version_most Python 3_8 Python 3_9  ... Python 3_5 or lower   
country_live  age                           ...                       
United States 18             10          4  ...                   0   
              21            100         21  ...                   0   
              30            116         17  ...                   3   
              40             57          5  ...                   3   
              50             47          2  ...                   1   
              60             14          3  ...                   0   

use_python_most                                                   \
python3_version_most Python 3_6 Python 3_7 Python 3_8 Python 3_9   
country_live  age                                                  
United States 18              1          1          8          5   
              21             15         15         19          5   
              30             10         25         23          7   
              40              7         12          8          7   
              50              3          4          9          0   
              60              5          5          5          4   

use_python_most          Web development                                   \
python3_version_most Python 3_5 or lower Python 3_6 Python 3_7 Python 3_8   
country_live  age                                                           
United States 18                       0          3          3         11   
              21                       3         33         64        103   
              30                       3         65         90        143   
              40                       1         16         34         73   
              50                       0         11         14         21   
              60                       0          3          8          8   

use_python_most                  
python3_version_most Python 3_9  
country_live  age                
United States 18              0  
              21              7  
              30             14  
              40             10  
              50              2  
              60              1  

[6 rows x 85 columns]

In [30]:
(
    pd.crosstab(index=[jb_2['country_live'], jb_2['age']], 
                columns=[jb_2['use_python_most'], 
                         jb_2['python3_version_most']]).loc[['United States'], ['Data analysis', 'Web development']]
)

use_python_most            Data analysis                                   \
python3_version_most Python 3_5 or lower Python 3_6 Python 3_7 Python 3_8   
country_live  age                                                           
United States 18                       0          1          5         10   
              21                       2         25         74        100   
              30                       3         34         83        116   
              40                       0         18         39         57   
              50                       2          7         33         47   
              60                       1          2         12         14   

use_python_most                     Web development                        \
python3_version_most Python 3_9 Python 3_5 or lower Python 3_6 Python 3_7   
country_live  age                                                           
United States 18              4                   0          3          3   
              21             21                   3         33         64   
              30             17                   3         65         90   
              40              5                   1         16         34   
              50              2                   0         11         14   
              60              3                   0          3          8   

use_python_most                             
python3_version_most Python 3_8 Python 3_9  
country_live  age                           
United States 18             11          0  
              21            103          7  
              30            143         14  
              40             73         10  
              50             21          2  
              60              8          1